In [33]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import seaborn as sns

In [34]:
%pip install gdown

Note: you may need to restart the kernel to use updated packages.


In [35]:
CARS_FILE_ID = '1liFEe1-yFISPSpRSvbv1wIH_avYNGmBI'
RANDOM_STATE = 42

random.seed(RANDOM_STATE)
np.random.seed(RANDOM_STATE)

!gdown --id {CARS_FILE_ID}

/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1liFEe1-yFISPSpRSvbv1wIH_avYNGmBI
From (redirected): https://drive.google.com/uc?id=1liFEe1-yFISPSpRSvbv1wIH_avYNGmBI&confirm=t&uuid=33c69fad-410a-4176-862a-498918c33127
To: /Users/ilolss/HSE/ml_hse/ml_project/part_6/dataset.csv
100%|██████████████████████████████████████| 1.01G/1.01G [02:02<00:00, 8.29MB/s]


In [36]:
df = pd.read_csv('dataset.csv')
print(df.shape)

(604047, 24)


In [37]:
df_with_na_column = df[df['engine_displacement'].isna()]
df_with_na_column

,production_year,mileage,condition,owners_number,pts_original,horse_power,accidents_resolution,region,seller_type,brand,...,engine_displacement,engine_power,fuel_rate,steering_wheel,price,price_segment,tags,auto_class,equipment,complectation_available_options
210905,2003,175000,CONDITION_OK,0,True,NaN,NaN,Чебаркуль,PRIVATE,Hyundai,...,NaN,NaN,NaN,LEFT,380000,MEDIUM,available_for_checkup;pts_original;real_photo;...,NaN,seats-5,NaN


In [38]:
df.drop(210905, inplace=True)

In [39]:
df['fuel_rate'].fillna(df['fuel_rate'].median(), inplace=True)

/var/folders/g6/kn9b03jj7fb6vtdn4q7l0_cm0000gn/T/ipykernel_73459/3003548066.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['fuel_rate'].fillna(df['fuel_rate'].median(), inplace=True)


In [40]:
df['pts_original'].fillna(True, inplace=True)
df['accidents_resolution'].fillna('OK', inplace=True)
df['auto_class'].fillna('NOT SPECIFIED', inplace=True)

/var/folders/g6/kn9b03jj7fb6vtdn4q7l0_cm0000gn/T/ipykernel_73459/2371127630.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['pts_original'].fillna(True, inplace=True)
/var/folders/g6/kn9b03jj7fb6vtdn4q7l0_cm0000gn/T/ipykernel_73459/2371127630.py:1: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['pts_or

In [41]:
df.drop('horse_power', axis=1, inplace=True)

In [42]:
def get_unique_values(series, sep=';'):
    uniq = set()
    for cell in series.dropna():
        for piece in cell.split(sep):
            s = piece.strip()
            if s:
                uniq.add(s)
    return np.array(list(uniq))

all_tags = get_unique_values(df['tags'])
all_options = get_unique_values(df['complectation_available_options'])
all_equipments = get_unique_values(df['equipment'])

In [43]:
def create_binary_features(df, column, unique_values, sep=';'):
    return (
        df[column]
        .str.get_dummies(sep=sep)
        .reindex(columns=unique_values, fill_value=0)
        .astype('int8')
    )

tags_dummies = create_binary_features(df, 'tags', all_tags)
options_dummies = create_binary_features(df, 'complectation_available_options', all_options)
equipment_dummies = create_binary_features(df, 'equipment', all_equipments)

In [44]:
tags_dummies = tags_dummies.astype('Sparse[int]')
options_dummies = options_dummies.astype('Sparse[int]')
equipment_dummies = equipment_dummies.astype('Sparse[int]')

In [45]:
full_df = pd.concat([
    df.drop(columns=['tags', 'complectation_available_options', 'equipment']),
    tags_dummies,
    options_dummies.drop(columns=['condition']),
    equipment_dummies.drop(columns=['condition'])
], axis=1)

In [46]:
duplicated_cols = full_df.columns[full_df.columns.duplicated()].unique()
for col in duplicated_cols:
    full_df[col] = full_df[col].astype("int8")

In [47]:
import numpy as np

names = full_df.columns[full_df.columns.duplicated()].unique()
new_cols = {}

for name in names:
    cols_i = [col for col in full_df.columns if col == name]
    sub_df = full_df.loc[:, cols_i].astype('int8')   # DataFrame shape (n_rows, k)
    
    # берём максимум по строкам → Series shape (n_rows,)
    max_series = sub_df.max(axis=1).astype('int8')
    
    new_cols[name] = max_series
    
    # удаляем *все* старые колонки с этим именем
    full_df.drop(columns=cols_i, inplace=True)

# теперь присваиваем новые колонки–Series (они одномерны!)
for name, s in new_cols.items():
    full_df[name] = s


/var/folders/g6/kn9b03jj7fb6vtdn4q7l0_cm0000gn/T/ipykernel_73459/333588981.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df[name] = s
/var/folders/g6/kn9b03jj7fb6vtdn4q7l0_cm0000gn/T/ipykernel_73459/333588981.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  full_df[name] = s
/var/folders/g6/kn9b03jj7fb6vtdn4q7l0_cm0000gn/T/ipykernel_73459/333588981.py:20: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider join

In [48]:
full_df.sample(10)

,production_year,mileage,condition,owners_number,accidents_resolution,region,seller_type,brand,model,body_type,...,body-kit,android-auto,wheel-power,high-beam-assist,black-roof,climate-control-1,ashtray-and-cigarette-lighter,17-inch-wheels,24-inch-wheels,front-seats-heat-vent
436947,2024,0,CONDITION_OK,0,OK,Санкт-Петербург,COMMERCIAL,Skoda,Karoq,ALLROAD_5_DOORS,...,0,0,1,0,0,1,0,0,0,0
329729,2020,40460,CONDITION_OK,1,ERROR,Москва,PRIVATE,Mercedes-Benz,E-Класс,COUPE_HARDTOP,...,0,0,0,0,1,1,1,0,0,0
343497,2000,380000,CONDITION_OK,4,OK,Москва,PRIVATE,Mercedes-Benz,M-Класс,ALLROAD_5_DOORS,...,0,0,0,0,0,0,0,0,0,0
439784,2019,170000,CONDITION_OK,1,ERROR,Москва,PRIVATE,Skoda,Octavia,LIFTBACK,...,0,1,0,0,0,1,1,1,0,0
550845,2023,107,CONDITION_OK,1,OK,Краснодар,PRIVATE,Lada (ВАЗ),Largus,WAGON_5_DOORS,...,0,0,1,0,0,0,0,0,0,0
431808,2015,110727,CONDITION_OK,1,OK,Уфа,COMMERCIAL,Renault,Sandero,HATCHBACK_5_DOORS,...,0,0,1,0,0,0,1,0,0,0
167063,2024,0,CONDITION_OK,0,OK,Химки,COMMERCIAL,Genesis,GV80 Coupe,ALLROAD_5_DOORS,...,0,1,1,1,1,1,0,0,0,1
391523,2022,13000,CONDITION_OK,0,OK,Владивосток,COMMERCIAL,Nissan,X-Trail,ALLROAD_5_DOORS,...,0,0,0,0,0,0,0,0,0,0
210052,2024,100,CONDITION_OK,1,OK,Новосибирск,PRIVATE,Hyundai,Elantra,SEDAN,...,0,0,0,0,0,0,0,0,0,0
46816,2024,34,CONDITION_OK,1,OK,Москва,COMMERCIAL,BMW,X5,ALLROAD_5_DOORS,...,0,0,0,0,0,0,0,0,0,0


In [49]:
import pandas as pd
import numpy as np
from scipy.cluster import hierarchy
from scipy.spatial.distance import squareform

numeric_df = full_df.select_dtypes(include=[np.number])
numeric_df = numeric_df.loc[:, numeric_df.nunique() > 1]

corr = numeric_df.corr().abs().fillna(0)

dist = 1 - corr

dist = (dist + dist.T) / 2
np.fill_diagonal(dist.values, 0)

dist = dist.clip(lower=0)

dist_vect = squareform(dist)
Z = hierarchy.linkage(dist_vect, method='average')

max_d = 0.4
clusters = hierarchy.fcluster(Z, t=max_d, criterion='distance')

cluster_df = pd.DataFrame({'feature': numeric_df.columns, 'cluster': clusters})
to_keep = []
for c in cluster_df['cluster'].unique():
    members = cluster_df.loc[cluster_df['cluster'] == c, 'feature']
    avg_corr = corr.loc[members, members].mean().sort_values(ascending=False)
    to_keep.append(avg_corr.index[0])
to_keep = list(dict.fromkeys(to_keep))

to_drop = [col for col in numeric_df.columns if col not in to_keep]
df_reduced = full_df.drop(columns=to_drop)

print(f"Удалено числовых коррелирующих признаков: {len(to_drop)}")
print(f"Осталось признаков в df_reduced: {df_reduced.shape[1]} из {full_df.shape[1]}")

Удалено числовых коррелирующих признаков: 115
Осталось признаков в df_reduced: 301 из 416


In [50]:
df_reduced.sample(10)

,mileage,condition,accidents_resolution,region,seller_type,brand,model,body_type,doors_count,seats,...,body-mouldings,steering-wheel-gear-shift-paddles,velvet-seats,body-kit,wheel-power,black-roof,ashtray-and-cigarette-lighter,17-inch-wheels,24-inch-wheels,front-seats-heat-vent
518746,33000,CONDITION_OK,ERROR,Аксай,PRIVATE,Lada (ВАЗ),2107,SEDAN,4,5,...,0,0,0,0,0,0,0,0,0,0
224119,23211,CONDITION_OK,OK,Москва,PRIVATE,Hyundai,Solaris,SEDAN,4,5,...,1,0,0,0,1,0,0,0,0,0
351020,6122,CONDITION_OK,OK,Уссурийск,COMMERCIAL,Mini,Countryman,ALLROAD_5_DOORS,5,5,...,0,0,0,0,0,0,0,0,0,0
424138,4500,CONDITION_OK,ERROR,Погар,PRIVATE,Renault,Kaptur,ALLROAD_5_DOORS,5,5,...,1,0,0,0,1,0,0,0,0,0
89776,245600,CONDITION_OK,OK,Тверь,PRIVATE,Chevrolet,Aveo,SEDAN,4,5,...,0,0,0,0,0,0,0,0,0,0
297634,18300,CONDITION_OK,OK,Москва,PRIVATE,Lexus,RX,ALLROAD_5_DOORS,5,5,...,1,1,0,0,1,0,0,0,0,1
449720,0,CONDITION_OK,OK,Москва,COMMERCIAL,Soueast,S07,ALLROAD_5_DOORS,5,5;7,...,0,1,0,0,1,0,0,0,0,1
49847,121000,CONDITION_OK,OK,Москва,PRIVATE,BMW,X6,ALLROAD_5_DOORS,5,5,...,0,1,0,1,0,1,1,0,0,0
48872,0,CONDITION_OK,OK,Краснодар,COMMERCIAL,BMW,X6,ALLROAD_5_DOORS,5,5,...,0,0,0,0,1,0,0,0,0,0
846,363500,CONDITION_OK,OK,Уфа,PRIVATE,Alfa Romeo,166,SEDAN,4,5,...,0,0,0,0,1,0,0,0,0,0


## Обучим LightGBM и CatBoost на df_reduced

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from lightgbm import LGBMRegressor
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [ ]:
X = df_reduced.drop('price', axis=1)
y = df_reduced['price']

In [ ]:
num_features = X.select_dtypes(include=['int64', 'float64', 'int8']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()

In [ ]:
preprocessor = ColumnTransformer(transformers=[
    ('num', StandardScaler(), num_features),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features),
])

**LightGBM**

In [ ]:
lgbm_pipeline = Pipeline([
    ('preproc', preprocessor),
    ('model', LGBMRegressor(objective='regression', random_state=42))
])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

In [ ]:
lgbm_pipeline.fit(X_train, y_train)

/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.600399 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6396
[LightGBM] [Info] Number of data points in the train set: 483236, number of used features: 2692
[LightGBM] [Info] Start training from score 2807543.358738


Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['mileage', 'doors_count',
                                                   'engine_displacement',
                                                   'engine_power', 'handling',
                                                   'safe_car', 'losangeles2017',
                                                   'catalog_china_landing_china_mark',
                                                   'catalog_photo', 'muscle',
                                                   'liquid',
                                                   'withNumericEnumFeaturePairs',
                                                   'catalog_china_landing_official_adaptation',
                                                   'user_reseller_7days',
                                                   '...
                                                   'certificate_manufacturer',
                                                   'jeneva2016',
                                                   'certificate_Favorit_Warranty', ...]),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['condition',
                                                   'accidents_resolution',
                                                   'region', 'seller_type',
                                                   'brand', 'model',
                                                   'body_type', 'seats',
                                                   'steering_wheel',
                                                   'price_segment',
                                                   'auto_class'])])),
                ('model',
                 LGBMRegressor(objective='regression', random_state=42))])

In [ ]:
y_pred_lgbm = lgbm_pipeline.predict(X_test)

/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [ ]:
mse_lgbm = mean_squared_error(y_test, y_pred_lgbm)
mae_lgbm = mean_absolute_error(y_test, y_pred_lgbm)
r2_lgbm  = r2_score(y_test, y_pred_lgbm)

In [ ]:
print("LightGBM:")
print(f"  MSE:  {mse_lgbm:.2f}")
print(f"  MAE:  {mae_lgbm:.2f}")
print(f"  R^2:  {r2_lgbm:.2f}")

LightGBM:
  MSE:  7009385199619.20
  MAE:  474050.08
  R^2:  0.75


**CatBoost**

In [ ]:
cat_pipeline = Pipeline([
    ('preproc', preprocessor),
    ('model', CatBoostRegressor(loss_function='RMSE', verbose=0, random_seed=RANDOM_STATE))
])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

In [ ]:
cat_pipeline.fit(X_train, y_train)

/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


Pipeline(steps=[('preproc',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['mileage', 'doors_count',
                                                   'engine_displacement',
                                                   'engine_power', 'handling',
                                                   'safe_car', 'losangeles2017',
                                                   'catalog_china_landing_china_mark',
                                                   'catalog_photo', 'muscle',
                                                   'liquid',
                                                   'withNumericEnumFeaturePairs',
                                                   'catalog_china_landing_official_adaptation',
                                                   'user_reseller_7days',
                                                   '...
                                                   'certificate_manufacturer',
                                                   'jeneva2016',
                                                   'certificate_Favorit_Warranty', ...]),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse_output=False),
                                                  ['condition',
                                                   'accidents_resolution',
                                                   'region', 'seller_type',
                                                   'brand', 'model',
                                                   'body_type', 'seats',
                                                   'steering_wheel',
                                                   'price_segment',
                                                   'auto_class'])])),
                ('model',
                 <catboost.core.CatBoostRegressor object at 0x161247100>)])

In [ ]:
y_pred_cat  = cat_pipeline.predict(X_test)

/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [ ]:
mse_cat = mean_squared_error(y_test, y_pred_cat)
mae_cat = mean_absolute_error(y_test, y_pred_cat)
r2_cat  = r2_score(y_test, y_pred_cat)

In [ ]:
print("LightGBM:")
print(f"  MSE:  {mse_lgbm:.2f}")
print(f"  MAE:  {mae_lgbm:.2f}")
print(f"  R^2:  {r2_lgbm:.2f}")

print("CatBoost:")
print(f"  MSE:  {mse_cat:.2f}")
print(f"  MAE:  {mae_cat:.2f}")
print(f"  R^2:  {r2_cat:.2f}")

LightGBM:
  MSE:  7009385199619.20
  MAE:  474050.08
  R^2:  0.75
CatBoost:
  MSE:  8188312842766.43
  MAE:  417499.35
  R^2:  0.71


## Попробуем подобрать гиперпараметры с помощью Bayesian Optimization

**LightGBM**

In [51]:
import optuna
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import r2_score
from lightgbm import LGBMRegressor

In [52]:
RANDOM_STATE = 42

In [53]:
X = df_reduced.drop('price', axis=1)
y = df_reduced['price']
X_train, X_valid, y_train, y_valid = train_test_split(
    X, y, test_size=0.2, random_state=RANDOM_STATE
)

num_features = X.select_dtypes(include=['int64','float64','int8']).columns.tolist()
cat_features = X.select_dtypes(include=['object']).columns.tolist()

preprocessor = ColumnTransformer([
    ('num', StandardScaler(), num_features),
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), cat_features),
])

In [54]:
X_train_proc = preprocessor.fit_transform(X_train)
X_valid_proc = preprocessor.transform(X_valid)

/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(
/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:921: UserWarning: pandas.DataFrame with sparse columns found.It will be converted to a dense numpy array.
  warnings.warn(


In [57]:
import lightgbm as lgb

def objective(trial):
    params = {
        'n_estimators':     trial.suggest_int('n_estimators', 100, 2000),
        'learning_rate':    trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
        'num_leaves':       trial.suggest_int('num_leaves', 31, 256),
        'max_depth':        trial.suggest_int('max_depth', 5, 30),
        'subsample':        trial.suggest_uniform('subsample', 0.5, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'reg_alpha':        trial.suggest_loguniform('reg_alpha', 1e-8, 10.0),
        'reg_lambda':       trial.suggest_loguniform('reg_lambda', 1e-8, 10.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 50),
        'random_state':     RANDOM_STATE,
        'verbose':          -1,
    }
    model = LGBMRegressor(**params)
    model.fit(
        X_train_proc, y_train,
        eval_set=[(X_valid_proc, y_valid)],
        callbacks=[lgb.early_stopping(stopping_rounds=50)]
    )
    preds = model.predict(X_valid_proc)
    return r2_score(y_valid, preds)

In [58]:
study = optuna.create_study(direction='maximize', study_name='lgbm_opt_r2')
study.optimize(
    objective,
    n_trials=50,
    n_jobs=-1,
    show_progress_bar=True,
)

print("Best R²:", study.best_value)
print("Best params:", study.best_params)

[I 2025-06-10 09:43:13,366] A new study created in memory with name: lgbm_opt_r2
  0%|          | 0/50 [00:00<?, ?it/s]/var/folders/g6/kn9b03jj7fb6vtdn4q7l0_cm0000gn/T/ipykernel_73459/1710893890.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate':    trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
/var/folders/g6/kn9b03jj7fb6vtdn4q7l0_cm0000gn/T/ipykernel_73459/1710893890.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample':        trial.suggest_uniform('subsample', 0.5, 1.0),
/var/folders/g6/kn9b03jj7fb6vtdn4q7l0_cm0000gn/T/ipykernel_73459/1710893890.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.

Training until validation scores don't improve for 50 rounds
Training until validation scores don't improve for 50 rounds
Training until validation scores don't improve for 50 rounds
Training until validation scores don't improve for 50 rounds
Training until validation scores don't improve for 50 rounds
Training until validation scores don't improve for 50 rounds
Training until validation scores don't improve for 50 rounds
Training until validation scores don't improve for 50 rounds
Training until validation scores don't improve for 50 rounds
Training until validation scores don't improve for 50 rounds
Training until validation scores don't improve for 50 rounds
Training until validation scores don't improve for 50 rounds
Early stopping, best iteration is:
[564]	valid_0's l2: 7.0614e+12


/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
Best trial: 2. Best value: 0.747859:   2%|▏         | 1/50 [22:04<18:01:55, 1324.81s/it]/var/folders/g6/kn9b03jj7fb6vtdn4q7l0_cm0000gn/T/ipykernel_73459/1710893890.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate':    trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
/var/folders/g6/kn9b03jj7fb6vtdn4q7l0_cm0000gn/T/ipykernel_73459/1710893890.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample':        trial.suggest_uniform('subsamp

[I 2025-06-10 10:05:18,118] Trial 2 finished with value: 0.7478592528022807 and parameters: {'n_estimators': 912, 'learning_rate': 0.06844899583344552, 'num_leaves': 181, 'max_depth': 5, 'subsample': 0.8942898882484294, 'colsample_bytree': 0.6031165535850378, 'reg_alpha': 0.2520936777453948, 'reg_lambda': 3.51357760339482e-06, 'min_child_weight': 31}. Best is trial 2 with value: 0.7478592528022807.
Did not meet early stopping. Best iteration is:
[481]	valid_0's l2: 8.56527e+12


/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
Best trial: 2. Best value: 0.747859:   4%|▍         | 2/50 [22:38<7:32:19, 565.41s/it]  /var/folders/g6/kn9b03jj7fb6vtdn4q7l0_cm0000gn/T/ipykernel_73459/1710893890.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'learning_rate':    trial.suggest_loguniform('learning_rate', 1e-3, 1e-1),
/var/folders/g6/kn9b03jj7fb6vtdn4q7l0_cm0000gn/T/ipykernel_73459/1710893890.py:9: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'subsample':        trial.suggest_uniform('subsamp

...

[I 2025-06-10 12:26:49,062] Trial 36 finished with value: 0.7592754069633155 and parameters: {'n_estimators': 1596, 'learning_rate': 0.004842247865373332, 'num_leaves': 107, 'max_depth': 27, 'subsample': 0.8467179252355471, 'colsample_bytree': 0.5115879279731358, 'reg_alpha': 0.10240305080763626, 'reg_lambda': 0.1371609257714269, 'min_child_weight': 19}. Best is trial 40 with value: 0.7778125884898708.


Best trial: 40. Best value: 0.777813:  98%|█████████▊| 49/50 [2:43:42<00:54, 54.77s/it]

[I 2025-06-10 12:26:55,844] Trial 37 finished with value: 0.7636245025078996 and parameters: {'n_estimators': 1647, 'learning_rate': 0.0048377350959822206, 'num_leaves': 128, 'max_depth': 27, 'subsample': 0.832995490079824, 'colsample_bytree': 0.5030219368974044, 'reg_alpha': 0.3134846652717163, 'reg_lambda': 0.07327730901442155, 'min_child_weight': 18}. Best is trial 40 with value: 0.7778125884898708.
Did not meet early stopping. Best iteration is:
[1670]	valid_0's l2: 6.7134e+12


/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(
Best trial: 40. Best value: 0.777813: 100%|██████████| 50/50 [2:44:39<00:00, 197.59s/it]

[I 2025-06-10 12:27:52,699] Trial 35 finished with value: 0.7602852339333606 and parameters: {'n_estimators': 1670, 'learning_rate': 0.004113125424886741, 'num_leaves': 192, 'max_depth': 27, 'subsample': 0.8744357434867911, 'colsample_bytree': 0.515274665210104, 'reg_alpha': 7.6015414896103835, 'reg_lambda': 0.10782316486854308, 'min_child_weight': 19}. Best is trial 40 with value: 0.7778125884898708.
Best R²: 0.7778125884898708
Best params: {'n_estimators': 1600, 'learning_rate': 0.04230796771458062, 'num_leaves': 109, 'max_depth': 26, 'subsample': 0.7744391789491678, 'colsample_bytree': 0.5013855637546327, 'reg_alpha': 0.08261606688965385, 'reg_lambda': 0.2539614097187018, 'min_child_weight': 18}


In [59]:
print("Best R²:", study.best_value)
print("Best params:", study.best_params)

Best R²: 0.7778125884898708
Best params: {'n_estimators': 1600, 'learning_rate': 0.04230796771458062, 'num_leaves': 109, 'max_depth': 26, 'subsample': 0.7744391789491678, 'colsample_bytree': 0.5013855637546327, 'reg_alpha': 0.08261606688965385, 'reg_lambda': 0.2539614097187018, 'min_child_weight': 18}


In [60]:
final_model = LGBMRegressor(**study.best_params, random_state=RANDOM_STATE)

In [65]:
import joblib

final_model.fit(X_train_proc, y_train)

joblib.dump(final_model, 'best_lightgbm_model.pkl')
joblib.dump(preprocessor, 'lightgbm_preprocessor.pkl')
print("Models saved to best_lightgbm_model.pkl and lightgbm_preprocessor.pkl")

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 1.508726 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6396
[LightGBM] [Info] Number of data points in the train set: 483236, number of used features: 2692
[LightGBM] [Info] Start training from score 2807543.358738
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
Models saved to best_lightgbm_model.pkl and lightgbm_preprocessor.pkl


In [66]:
y_pred = final_model.predict(X_valid_proc)

/Users/ilolss/anaconda3/envs/ml-project-2-course/lib/python3.10/site-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [64]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

print(f"Test MAE: {mean_absolute_error(y_valid, y_pred):.4f}")
print(f"Test MSE: {mean_squared_error(y_valid, y_pred):.4f}")
print(f"Test R^2: {r2_score(y_valid, y_pred):.4f}")

Test MAE: 283821.9014
Test MSE: 6138210709233.8760
Test R^2: 0.7808


### Выводы

$R^2 = 0.7808$

In [ ]:
param = {
    'n_estimators': 1600,
    'learning_rate': 0.04230796771458062, 
    'num_leaves': 109, 
    'max_depth': 26, 
    'subsample': 0.7744391789491678, 
    'colsample_bytree': 0.5013855637546327, 
    'reg_alpha': 0.08261606688965385, 
    'reg_lambda': 0.2539614097187018, 
    'min_child_weight': 18
}